In [1]:
using Pkg
Pkg.activate(".")
Pkg.add(url="https://github.com/MonssafToukal/SolverParameters.jl", rev="main")
Pkg.add(url="https://github.com/ProofOfConceptForJuliSmoothOptimizers/BBModels.jl", rev="main")

  Activating project at `C:\Users\tangi\.julia\dev\BBModels.jl\repeat`
    Updating git-repo `https://github.com/MonssafToukal/SolverParameters.jl`
    Updating registry at `C:\Users\tangi\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\tangi\.julia\dev\BBModels.jl\repeat\Project.toml`
  No Changes to `C:\Users\tangi\.julia\dev\BBModels.jl\repeat\Manifest.toml`
    Updating git-repo `https://github.com/ProofOfConceptForJuliSmoothOptimizers/BBModels.jl`
   Resolving package versions...
  No Changes to `C:\Users\tangi\.julia\dev\BBModels.jl\repeat\Project.toml`
  No Changes to `C:\Users\tangi\.julia\dev\BBModels.jl\repeat\Manifest.toml`


In [2]:
using SolverParameters

For each solver, we define an instance of an `AbstractParameterSet` with the tunable parameters.
See [https://monssaftoukal.github.io/SolverParameters.jl/dev/](https://monssaftoukal.github.io/SolverParameters.jl/dev/) for an example.

In [3]:
struct LBFGSParameterSet{T <: Real} <: AbstractParameterSet
  mem::Parameter{Int, IntegerRange{Int}}
  τ₁::Parameter{T, RealInterval{T}}
  bk_max::Parameter{Int, IntegerRange{Int}}
  # add scaling
  
  function LBFGSParameterSet{T}(;mem::Int = 5, τ₁::T = T(0.9999), bk_max::Int = 25) where {T}        
    p_set = new(
      Parameter(mem, IntegerRange(Int(1), Int(20)), "mem"),
      Parameter(τ₁, RealInterval(T(0), T(1)), "τ₁"),
      Parameter(bk_max, IntegerRange(Int(10), Int(50)), "bk_max"),
    )
    return p_set
  end

  function LBFGSParameterSet(;kwargs...)
    return LBFGSParameterSet{Float64}(; kwargs...)
  end
end

In [4]:
include("lbfgs.jl") # essentially copy-paste from JSOSolvers.jl/src/lbfgs.jl
#=
SolverCore.solve!(
  solver::LBFGSSolver{T, V},
  param::LBFGSParameterSet,
  nlp::AbstractNLPModel{T, V},
  stats::GenericExecutionStats{T, V};
  kwargs...,
)
=#

Let us now illustrate how we can use `BBModels` to model the problem of parameter tuning, cf. https://proofofconceptforjulismoothoptimizers.github.io/BBModels.jl/dev/.

In [5]:
using BBModels

We select a set of problems.

In [6]:
using ADNLPModels, OptimizationProblems, OptimizationProblems.ADNLPProblems
n = 5
meta = OptimizationProblems.meta
list = meta[
  meta.minimize .& (meta.ncon .== 0) .& .!meta.has_bounds .& (20 .≤ meta.nvar .≤ 100),
  :name
]
problems = [eval(p)() for (_, p) ∈ zip(1:n, Symbol.(list))];
nvars = [p.meta.nvar for p in problems]

5-element Vector{Int64}:
  91
 100
 100
 100
 100

We instantiate a `BBModel` as follows.

In [7]:
? BBModel

search: BBModel BBModels BBModelMeta



Mutable struct `BBModel`

Represents a black box optimization problem that follows the NLPModel API.

The following constructors are available:

```
BBModel(parameter_set, problems, solver_function, f; kwargs...)
BBModel(parameter_set, problems, solver_function, f, c, lcon, ucon; kwargs...)
```

  * `parameter_set::AbstractParameterSet`: structure containing parameters information;
  * `problems::Vector{AbstractNLPModel}`: set of problem to run the benchmark on;
  * `solver_function::Function`: function that takes an `AbstractNLPModel` and a `AbstractParameterSet` and returns a [`GenericExecutionStats`](https://github.com/JuliaSmoothOptimizers/SolverCore.jl/blob/main/src/stats.jl).
  * `f::Function`: Given a `Vector{ProblemMetrics}` returns a score as a Float64 (examples are [`time_only`](@ref), [`memory_only`](@ref), [`sumfc`](@ref));

For constrained problems:

```
lcon ≤ c(x) ≤ ucon
```

  * `c::Function`: constraint function;
  * `lcon::AbstractVector`: lower bound on the constraint;
  * `ucon::AbstractVector`: upper bound on the constraint.

Additional keyword arguments are:

  * `subset::NTuple{N, Symbol}`: subset of parameters to be considered (by default all parameters from `parameter_set`);
  * `x0::AbstractVector`: initial values for the parameters (by default `Float64.(values(parameter_set))`);
  * `lvar::AbstractVector`: lower bound on the the parameters (by default `Float64.(lower_bounds(parameter_set))`);
  * `uvar::AbstractVector`: upper bound on the the parameters (by default `Float64.(lower_bounds(parameter_set))`);
  * `name::String`: name of the problem (by default: "Generic").

Note that if `x0` is not provided, the computations are run in `Vector{Float64}`.


In [8]:
param_set = LBFGSParameterSet()
solver_func = lbfgs
subset = (:mem, ) # optimize only `mem`

using BenchmarkTools
# cost function to be optimized:
function fun(vec_metrics::Vector{ProblemMetrics})
  penalty = 1e2
  global fx = 0
  for p in vec_metrics
    failed = is_failure(BBModels.get_status(p))
    nobj = get_counters(p).neval_obj
    med_time = BenchmarkTools.median(get_times(p))
    @info "Problem $(p.pb_id) - $(problems[p.pb_id].meta.name) : $(failed) #f:$(nobj) t=$(med_time)"
    fx += failed * penalty + nobj + med_time
  end
  return fx
end

model = BBModel(
  param_set, # AbstractParameterSet
  problems, # vector of AbstractNLPModel
  solver_func, # (::AbstractNLPModel, ::AbstractParameterSet) -> GenericExecutionStats
  fun, # time_only, memory_only, sumfc OR a hand-made function
  subset = subset,
)

BBModel - Black Box Optimization Model
  Problem name: generic-BBModel
   All variables: ████████████████████ 1      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ████████████████████ 1              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   1               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: (------% sparsity)         

  Counters:
      

In [9]:
function very_smart_algo(model::BBModel; verbose = 0, max_time = 30.0, max_iter = 10)
  cache = Float64[]
  cache_x = Any[]
  start_time = time()
  for i=1:max_iter
    time() - start_time > max_time && break
    x = SolverParameters.rand(model.subset, model.parameter_set) # may return categorical variables
    @show x
    fx = BBModels.obj_cat(model, x)
    push!(cache, fx)
    push!(cache_x, x)
    (verbose > 0) && println("$i: fx=$fx")
  end
  is = argmin(cache)
  println("Best value is mem=$(cache_x[is])")
  return cache_x[is]
end

very_smart_algo (generic function with 1 method)

In [10]:
vals = very_smart_algo(model, verbose = 1)

x = Any[16]
1: fx=432.0054143939394
x = Any[4]

[ Info: Problem 1 - NZF1 : false #f:274 t=0.0006509641873278237
[ Info: Problem 2 - arglina : false #f:8 t=0.0008424931129476584
[ Info: Problem 3 - arglinb : false #f:56 t=0.0012521005509641874
[ Info: Problem 4 - arglinc : false #f:56 t=0.0013511019283746557
[ Info: Problem 5 - arwhead : false #f:38 t=0.0013177341597796143



2: fx=582.0053828856749
x = Any[6]

[ Info: Problem 1 - NZF1 : false #f:426 t=0.0008281680440771351
[ Info: Problem 2 - arglina : false #f:8 t=0.0008613636363636363
[ Info: Problem 3 - arglinb : false #f:56 t=0.0012339876033057853
[ Info: Problem 4 - arglinc : false #f:56 t=0.0013073002754820937
[ Info: Problem 5 - arwhead : false #f:36 t=0.0011520661157024795



3: fx=528.0053011707989
x = Any[19]

...


4: fx=360.0050409435261
x = Any[16]


[ Info: Problem 1 - NZF1 : false #f:202 t=0.0004784090909090909
[ Info: Problem 2 - arglina : false #f:8 t=0.0008509297520661157
[ Info: Problem 3 - arglinb : false #f:56 t=0.0012805785123966942
[ Info: Problem 4 - arglinc : false #f:56 t=0.0012649104683195593
[ Info: Problem 5 - arwhead : false #f:38 t=0.001166115702479339


5: fx=432.0053504132231
Best value is mem=Any[19]


[ Info: Problem 1 - NZF1 : false #f:274 t=0.0006350895316804408
[ Info: Problem 2 - arglina : false #f:8 t=0.0009116391184573004
[ Info: Problem 3 - arglinb : false #f:56 t=0.0014217630853994492
[ Info: Problem 4 - arglinc : false #f:56 t=0.001253409090909091
[ Info: Problem 5 - arwhead : false #f:38 t=0.001128512396694215


1-element Vector{Any}:
 19

In [11]:
set_values!(subset, param_set, vals)

In [12]:
value(param_set.mem)

19

To avoid instantiating all the problems from the start, it is also possible to pass a list of functions:

In [13]:
problems_expr = [() -> eval(p)() for (_, p) ∈ zip(1:n, Symbol.(list))]
model_expr = BBModel(
  param_set, # AbstractParameterSet
  problems_expr, # vector of Function
  solver_func, # (::AbstractNLPModel, ::AbstractParameterSet) -> GenericExecutionStats
  fun, # time_only, memory_only, sumfc OR a hand-made function
  subset = subset,
)

BBModel - Black Box Optimization Model
  Problem name: generic-BBModel
   All variables: ████████████████████ 1      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ████████████████████ 1              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   1               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: (------% sparsity)         

  Counters:
      

In [14]:
vals = very_smart_algo(model_expr, verbose = 1)

x = Any[11]
1: fx=476.0054148415978
x = Any[11]


[ Info: Problem 1 - NZF1 : false #f:318 t=0.0007369490358126723
[ Info: Problem 2 - arglina : false #f:8 t=0.0008492079889807164
[ Info: Problem 3 - arglinb : false #f:56 t=0.0012604338842975207
[ Info: Problem 4 - arglinc : false #f:56 t=0.0013162190082644629
[ Info: Problem 5 - arwhead : false #f:38 t=0.0012520316804407715


2: fx=476.0052329889807
x = Any[19]

[ Info: Problem 1 - NZF1 : false #f:318 t=0.0006868457300275482
[ Info: Problem 2 - arglina : false #f:8 t=0.0008461776859504133
[ Info: Problem 3 - arglinb : false #f:56 t=0.001206714876033058
[ Info: Problem 4 - arglinc : false #f:56 t=0.0012510674931129478
[ Info: Problem 5 - arwhead : false #f:38 t=0.0012421831955922865



3: fx=360.0050376721763
x = Any[2]


...

7: fx=420.0052108471075
x = Any[6]


[ Info: Problem 1 - NZF1 : false #f:262 t=0.0006767217630853995
[ Info: Problem 2 - arglina : false #f:8 t=0.0008578168044077135
[ Info: Problem 3 - arglinb : false #f:56 t=0.001250275482093664
[ Info: Problem 4 - arglinc : false #f:56 t=0.0012604683195592285
[ Info: Problem 5 - arwhead : false #f:38 t=0.0011655647382920109


8: fx=528.0053217630854
Best value is mem=Any[19]


[ Info: Problem 1 - NZF1 : false #f:370 t=0.0007869834710743802
[ Info: Problem 2 - arglina : false #f:8 t=0.0009134641873278237
[ Info: Problem 3 - arglinb : false #f:56 t=0.001191322314049587
[ Info: Problem 4 - arglinc : false #f:56 t=0.0012330922865013774
[ Info: Problem 5 - arwhead : false #f:38 t=0.001196900826446281


1-element Vector{Any}:
 19

# How to integrate this in JSO?

The first goal is to model the problem of parameter tuning:
- Step 1: Add `SolverParameters.jl` somewhere: either in SolverCore.jl or independent package;
- Step 2-3: use the structure defined in `SolverParameters.jl` for the JSO-solvers (right now each solver has its own structure or no structure);
- Step 2-3: Register `BBModels.jl`.

Then, the next goal is to decide how to solve it, for instance https://github.com/ProofOfConceptForJuliSmoothOptimizers/SolverTuning.jl.
Note that this will also be used for algorithm selection in `JSOSuite.jl`.